In [5]:
from shapely.geometry import LineString
import matplotlib.pyplot as plt
import casadi as ca

In [8]:
no_states = 4
no_inputs = 2
predict_horizon = 4
dt = 0.2
state = ca.MX.sym('state', no_states)
control = ca.MX.sym('control', no_inputs)

print("State shape:", state.shape)
print("Control shape:", control.shape)

line_points = [(0,0), (20,0), (40,0)]
reference_line = LineString(line_points)

State shape: (4, 1)
Control shape: (2, 1)


In [ ]:
# Kinematic Bicycle Model
# https://thomasfermi.github.io/Algorithms-for-Automated-Driving/Control/BicycleModel.html


def vehicle_model(state, control):
    x = state[0] # X position
    y = state[1] # Y position
    theta = state[2] # Heading angle
    v = state[3] # Velocity

    delta = control[0]  # Steering angle
    a = control[1]      # Acceleration

    dx = v * ca.cos(theta)
    dy = v * ca.sin(theta)
    dtheta = (v * ca.tan(delta)) / 2.5  # Assuming wheelbase of 2.5m
    dv = a

    return ca.vertcat([dx, dy, dtheta, dv])


def rk2(state, u):
    k1 = dt * vehicle_model(state, u)
    k2 = dt * vehicle_model(state + 0.5 * k1, u)
    next_state = state + k2
    return next_state

def cost_function(state, control):
    # x = state[0]
    y = state[1]
    # theta = state[2]
    # v = state[3]

    delta = control[0]  
    a = control[1]       

    cost = 0.0

    cost += y**2 
    cost += 0.1 * delta**2 + 0.1 * a**2
    return cost